In [2]:
import pandas as pd
import numpy
from __future__ import division

In [3]:
df3_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment3_WS4_has_duration.csv")

In [4]:
df3_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1022190|362029|564420,Release Offer,Release Offer,18/7/2018 10:30:26,CHRISTINAA,Faculty,7,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MCW,HDCW,Full Degree,AF,AS,Y,$ACC,Y,54 days 7 hours 3 minutes 37 seconds,4691017.0


In [5]:
numOfWorkflowProcesses = df3_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [6]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [7]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [8]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [10]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS4T[df3_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")


In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)


In [15]:
s3_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s3_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,57,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Release Offer}
1,2,142,28 days 6 hours 15 minutes 55 seconds,9.0 days 3.0 hours 30.0 minutes 22.5 seconds,40.49%,"{OC - Reviewed - Other, Defer, Decline, AR - S..."
2,3,32,51 days 19 hours 18 minutes 18 seconds,26.0 days 2.0 hours 59.0 minutes 7.5 seconds,28.13%,"{OC - Reviewed - Other, Decline, OC - Not Sati..."
3,4,12,48 days 20 hours 4 minutes 45 seconds,26.0 days 2.0 hours 45.0 minutes 10.5 seconds,18.75%,"{Send for Assessment, Decline, Back to Perform..."
4,5,3,76 days 17 hours 26 minutes 35 seconds,43.0 days 6.0 hours 55.0 minutes 6.0 seconds,20.0%,"{Defer, Assessment Decision - Qualified, OC - ..."
5,6,3,94 days 7 hours 21 minutes 56 seconds,59.0 days 2.0 hours 41.0 minutes 19.0 seconds,5.56%,"{Decline, Assessment Decision - No Decision, A..."
6,7,1,40 days 2 hours 45 minutes 12 seconds,40.0 days 2.0 hours 45.0 minutes 12.0 seconds,14.29%,"{Decline, AR - Satisfied - previous or current..."


In [16]:
s3_t4.to_csv('segment3_WS4_table.csv', index=False)


